In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Overview of Assignment** : The below activity has been done to classify bird species. Convolutional Neural Network was build and Data Augmntation , Batch Normalization , Dropout , Max Pooling techniques were used to improve the accuracy of the model.Adam Optimizer and RMSprop Optimizer were used to monitor the change in accuracy with two optimizers.Transfer learning was used in the project and InceptionV3 model was used.

**Description of Data** :
The "100-bird-species" data contains images of 230 bird species. The data is split into test, validation, and train data with 1150 images, 1150 images, and 32025 images, respectively. The images are all in jpg format. The dimesions of the images are 224, 224, 3.

**Summary of Methods** : All the required libraries were loaded and data was saved in three different directories namely Train directory(images for training model) , Validation Directory(images for validation) and Test directory(images for testing the accuracy of model).ImageDataGenerator data augmentation was  used to scale the images. InceptionV3 was applied to the ImageNet dataset for training. Adam and RMSprop optimizers were used.Functional API was used as well to classify the birds.

****Summary of Model: **** 

First Model : The First model was build using convolutional 2D layers and using Batch Normalization, Droput , Maxpooling . Adam optimizer was used with a learning rate 0.001.Layers were flattened and relu activation function was used. The dense layer output was 230 equal to the number of birds categories and softmax activation function was used

Second Model: Functional API was used in this model for bird classification. 4 Towers were created with different 2D Convolutional layers. Average Pooling , Dropout techniques were used. Adam Optimizer with learning rate 0.001 was used. Relu and Softmax activation functions were used.

Third Model : Transfer learning was used in this model . InceptionV3 was applied on imagenet and the layers were freeezed. Adam optimizer was used with learning rate 0.001. Batch Normalization and Dropout techniques were applied.

Fourth Model and Fifth Model: InceptionV3 was used on imagenet dataset.RMSprop optimizer was used with learning rate 0.001.Fourth model had all the layers freezed and 5th model had all the layers freezed except for bottom 4 layers.

Sixth Model: The model was similar to fouth model but this time the images in the train and validation directory were augmented applying different zoom , hight width changes.

**Analysis of Result **: 

First Model : The first model test accuracy was only 0.6% . 
Second Model : The second model test accuracy was 0.3% and loss on the training and validation set were 5.5

Third Model : The Third model gave test accuracy of 93.96% and the loss on train and validation data was ~0.49

Fourth Model and Fifth Model : The Fourth model gave test accuracy of 90.73% and fifth model gave the test accuracy 91.5%. The loss observed was slightly less in the fifth model. Model with freezing all the layers except bottom 4 layers performed better than the model where all the layers were freezed

Sixth Model: The model gave 86.5% accuracy and the losses were slightly higher compared to Fourth and Fifth model


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend, models, layers, regularizers , optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization , Concatenate
from keras.layers import Activation, Flatten, Dense, Dropout ,Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os
import numpy as np
import pandas as pd

#Directory for the Train data , Validation data and test data

In [ ]:
train_directory ='../input/100-bird-species/train'
val_directory = '../input/100-bird-species/valid'
test_directory = '../input/100-bird-species/test'

ImageDataGenerator API in Keras  is used to create augmented image generator. ImageDataGenerator generates batches of image data with real-time data augmentation.Image augmentation artificially creates training images through different ways of processing or combination of multiple processing, such as random rotation, shifts, shear and flips, etc.

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255)
val_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

****Create Data Sets****

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_directory, # The directory where the train data is located
    target_size=(224, 224), # Reshape the image to 224 by 224 pixels. This is important because it makes sure all images are the same size.
    batch_size=32, # We will take images in batches of 32.
    color_mode='rgb',
    class_mode='sparse') # The classification is not binary.

validation_generator = train_datagen.flow_from_directory(
    val_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse')

In [ ]:
train_generator.class_indices

In [ ]:
num_classes=(len(train_generator.class_indices))
print(num_classes)

**Visualizing 25 Bird Species******

In [ ]:
def display_random_grid(ncols=5, ds_path=train_directory):
    fig, ax = plt.subplots(ncols=ncols, nrows=ncols, figsize=(15, 15))
    
    for i in range(ncols):
        for j in range(ncols):
            bird_species = random.choice(os.listdir(ds_path))
            random_bird_path = random.choice(os.listdir(ds_path + '/'+ bird_species))
            random_bird = mpimg.imread(ds_path + '/' + bird_species + '/' + random_bird_path)
            ax[i, j].imshow(random_bird)
            ax[i, j].set_title(bird_species)
            ax[i, j].axis('off')
            
display_random_grid()

#Creating a  Convolutional Network using Batch Normalization and Droput. Very less accuracy was seen with the below model

In [ ]:
backend.clear_session()
model_base = models.Sequential()
model_base.add(layers.Conv2D(512, (3,3), activation = 'relu', input_shape = (224, 224, 3)))
model_base.add(layers.MaxPool2D((2,2)))
model_base.add(BatchNormalization())
model_base.add(layers.Conv2D(512, (3,3), activation = 'relu'))
model_base.add(layers.MaxPool2D((2,2)))
model_base.add(BatchNormalization())
model_base.add(layers.Conv2D(256, (3,3), activation = 'relu'))
model_base.add(layers.MaxPool2D((2,2)))
model_base.add(BatchNormalization())
model_base.add(layers.Flatten())
model_base.add(layers.Dense(128, activation='relu'))
model_base.add(layers.Dropout(0.5))
model_base.add(layers.Dense(num_classes, activation='softmax'))

model_base.compile(optimizer =optimizers.Adam(lr=0.001),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

history = model_base.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=20,
    verbose = 1,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights= True)])

test_loss, test_acc = model_base.evaluate_generator(test_generator, steps = 20)
print('test_acc:', test_acc)

Using Functional API for bird classification

In [ ]:
backend.clear_session()
visible = Input(shape=(224,224,3))
#Create Tower1
conv_1_1 = Conv2D(64, (1,1), padding= 'same' , activation= 'relu', strides=2)(visible)
#Create Tower2
conv_1_2 = Conv2D(64, (1,1), padding= 'same' , activation = 'relu')(visible)
conv_2_2 = Conv2D(64, (3,3), padding= 'same' , activation= 'relu', strides=2)(conv_1_2)
#Create Tower3
conv_1_3 = AveragePooling2D((3,3), padding= 'same', strides=2)(visible)
conv_2_3 = Conv2D(64, (3,3), padding= 'same' , activation= 'relu')(conv_1_3)
#Create Tower4
conv_1_4 = Conv2D(64, (1,1), padding= 'same' , activation = 'relu')(visible)
conv_2_4 = Conv2D(64, (3,3), padding= 'same' , activation = 'relu')(conv_1_4)
conv_3_4 = Conv2D(64, (3,3), padding= 'same' , activation = 'relu', strides=2)(conv_2_4)

# Concatentate
merge = Concatenate(axis=-1)([conv_1_1, conv_2_2, conv_2_3, conv_3_4])

# Flatten into fully connected layer
flat = Flatten()(merge)

# Hidden connected layer and output
hidden = Dense(32, activation='relu')(flat)
drop = Dropout(0.5)(hidden)
output = Dense(num_classes, activation='softmax')(drop)

model_birdcl= Model(inputs=visible, outputs=output)


model_birdcl.compile(optimizer = optimizers.Adam(lr=0.001),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])

history = model_birdcl.fit_generator(
          train_generator,
          steps_per_epoch=1001,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=29,
          verbose=1, 
          callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

test_loss , test_acc = model_birdcl.evaluate_generator(test_generator , steps = 29)

print('Using Functional API and Adam Optimizer the accuracy is ---', test_acc)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plot_model(model_birdcl)

#Using Transfer learning model InceptionV3 for Bird Classification.Optimizer used is Adam

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.

In [ ]:
backend.clear_session()
incbasemodel4 = InceptionV3(weights = 'imagenet' , include_top = False, input_shape = (224 , 224 , 3))
incbasemodel4.trainable = False
modelinceptionv3_vers4 = models.Sequential()
modelinceptionv3_vers4.add(incbasemodel4)
modelinceptionv3_vers4.add(layers.Conv2D(1024 , (3,3) , padding = 'same', activation = 'relu'))
modelinceptionv3_vers4.add(BatchNormalization())
modelinceptionv3_vers4.add(layers.Dropout(0.5))
modelinceptionv3_vers4.add(layers.Flatten())
modelinceptionv3_vers4.add(layers.Dense(512 , activation = 'relu'))
modelinceptionv3_vers4.add(layers.Dropout(0.5))
modelinceptionv3_vers4.add(layers.Dense(num_classes, activation = 'softmax'))
modelinceptionv3_vers4.compile(optimizer = optimizers.Adam(lr=0.001),
               loss = 'sparse_categorical_crossentropy',
               metrics = ['accuracy'])
history = modelinceptionv3_vers4.fit_generator(
          train_generator,
          steps_per_epoch=1001,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=29,
          verbose=1, 
          callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])
test_loss , test_acc = modelinceptionv3_vers4.evaluate_generator(test_generator , steps = 29)
print('Using InceptionV3 Model and Adam Optimizer the accuracy is ---', test_acc)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#Using Transfer learning model InceptionV3 for Bird Classification.Optimizer used is RMSprop

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem

In [ ]:
backend.clear_session()
conv_base = InceptionV3 (weights = 'imagenet', 
                  include_top = False,
                  input_shape = (224, 224, 3))
conv_base.trainable = False # Freeze the Inception V3 weights.

model = models.Sequential()
model.add(conv_base)
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(optimizer = optimizers.RMSprop(lr=0.001, decay=1e-6, momentum=0.9),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
history = model.fit_generator(
          train_generator,
          steps_per_epoch=1001,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=29,
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

test_loss , test_acc = model.evaluate_generator(test_generator , steps = 29)

print('InceptionV3 Model and using RMSprop optimizer accuracy is ----', test_acc)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#Using Transfer learning model InceptionV3 for Bird Classification.Optimizer used is RMSprop. Freezing weights except for bottom 4 layers

In [ ]:
backend.clear_session()
conv_basev2 = InceptionV3 (weights = 'imagenet', 
                  include_top = False,
                  input_shape = (224, 224, 3))
conv_basev2.trainable = False # Freeze the Inception V3 weights.
for layer in conv_basev2.layers[:-4]:
  trainable = False
for layer in conv_basev2.layers:
  print(layer, layer.trainable)
model = models.Sequential()
model.add(conv_basev2)
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(optimizer = optimizers.RMSprop(lr=0.001, decay=1e-6, momentum=0.9),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
history = model.fit_generator(
          train_generator,
          steps_per_epoch=1001,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=29,
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

test_loss , test_acc = model.evaluate_generator(test_generator , steps = 29)

print('InceptionV3 freezing all weights but bottom 4 layers accuracy is -----', test_acc)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#Using Data augmentation to improve accuracy.
#Data augmentation allows us to randomally transform images before sending them to the model for training.  
#The random transformation changes the images into 'new' images and allows for an increase in traning data without have additional images. 

In [ ]:
train_datagen2 = ImageDataGenerator(
    rescale=1./255,# The image augmentaion function in Keras
    rotation_range=40, # Rotate the images randomly by 40 degrees
    width_shift_range=0.2, # Shift the image horizontally by 20%
    height_shift_range=0.2, # Shift the image veritcally by 20%
    zoom_range=0.2, # Zoom in on image by 20%
    horizontal_flip=True, # Flip image horizontally 
    fill_mode='nearest') # How to fill missing pixels after a augmentaion opperation


test_datagen2 = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator2 = train_datagen2.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse')

validation_generator2 = train_datagen2.flow_from_directory(
    val_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse')

test_generator2 = test_datagen2.flow_from_directory( # Resize test data
    test_directory,
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode='sparse')


In [ ]:
backend.clear_session()
conv_base = InceptionV3 (weights = 'imagenet', 
                  include_top = False,
                  input_shape = (224, 224, 3))
conv_base.trainable = False # Freeze the Inception V3 weights.

model = models.Sequential()
model.add(conv_base)
model.add(BatchNormalization())
model.add(layers.Flatten())
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(optimizer = optimizers.RMSprop(lr=0.001, decay=1e-6, momentum=0.9),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
history = model.fit_generator(
          train_generator2,
          steps_per_epoch=1001,
          epochs=50,
          validation_data=validation_generator2,
          validation_steps=29,
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

test_loss , test_acc = model.evaluate_generator(test_generator2 , steps = 29)

print('InceptionV3_Vers3 adding Droput and Normalization accuracy', test_acc)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()